# CUDA 数组接口

由于将数据从 CPU 移动到 GPU 的成本很高，我们希望始终在 GPU 上保持尽可能多的数据。

有时在我们的工作流程中，我们也想要更换使用的工具。比如我们用 `cupy` 加载了一个数据数组，但我们想用 `numba` 编写自定义的 CUDA 内核。或者我们想切换到使用像 `pytorch` 这样的深度学习框架。

当这些库中的任何一个将数据加载到 GPU 上时，内存中的数组基本上是相同的。cupy 的 `ndarray` 和 numba 的 `DeviceNDArray` 之间的区别仅仅在于该数组如何被包装并连接到 Python。

幸运的是，通过像 [DLPack](https://github.com/dmlc/dlpack) 和 [`__cuda_array__interface__`](https://numba.readthedocs.io/en/stable/cuda/cuda_array_interface.html) 这样的工具，我们可以在不修改 GPU 上的数据的情况下从一种类型转换为另一种类型。我们只需创建一个新的 Python 包装器对象并传输所有设备指针。

确保流行的 GPU Python 库之间的兼容性是 RAPIDS 社区的核心目标之一。

![](images/array-interface.png)

让我们看看这个实际操作！

我们首先用 CuPy 创建一个数组。

In [ ]:
import cupy as cp
cp_arr = cp.random.random((1, 100_000, 10_000))
cp_arr

In [ ]:
type(cp_arr)

现在让我们将其转换为 Numba 数组。

In [ ]:
from numba import cuda
numba_arr = cuda.to_device(cp_arr)
numba_arr

_注意 GPU 内存使用量保持不变。这是因为 `cp_arr` 和 `numba_arr` 引用相同的底层数据数组，但类型不同。_

我们还可以将数组转换为 PyTorch 的 `Tensor` 对象。

In [ ]:
import torch  # 需要安装 pytorch

In [ ]:
torch_arr = torch.as_tensor(numba_arr, device='cuda')
torch_arr

In [ ]:
type(torch_arr)